# Рубежная контрольная работа

Эту работу выполнять не обязательно. Баллы, набранные за работу, просуммируются с баллами, которые были набраны за контрольную работу 1, и будут выставлены как оценка за промежуточную аттестацию "автоматом".

Сделайте копию ноутбука.

Присвойте переменной `v` значение - свой табельный номер ИСУ, это нужно для определения вашего варианта. Запустите код в следующей ячейке.

In [ ]:
from datetime import datetime
current_datetime = datetime.now()
print(current_datetime)
v = 336071 # по умолчанию вписан мой номер

2023-12-15 08:10:25.801246


# Общее задание

Нужно решить задачу бинарной классификации, предварительно построив признаковое описание объектов на основе нескольких таблиц.

Целевая переменная - пол клиента.

В качестве модели нужно использовать нейронную сеть, которую нужно строить с помощью `keras` или `torch` на выбор студента.

# Данные

Для всех последующих заданий будем использовать обезличенные транзакционные банковские данные. Для этого считайте в переменные **transactions, tr_mcc_codes и gender_train** из одноимённых таблиц из папки data. Для таблицы transactions используйте только первые n=1000000 строк.

Эти данные уже использовались в работе 2. Описание данных тут: https://drive.google.com/file/d/13sn8orjkoGfO1SVZYByXNmZQYFRfmOdp/view?usp=sharing

Скачать данные отсюда: https://drive.google.com/drive/folders/1YAMe7MiTxA-RSSd8Ex2p-L0Dspe6Gs4L?usp=sharing

## Задание 1 (6 баллов)

В задании требуется на основе нескольких таблиц с данными сделать признаковое описание объектов.

Объектами являются клиенты. Клиенты идентифицируются с помощью `customer_id`, которые есть в таблицах **transactions** и **gender_train**. В качестве призаков нужно использовать даннее по категориям транзаций: ```mcc_code``` — mcc-код транзакции есть в таблицах **transactions** и **tr_mcc_codes**.


Для выбора варианта запустите код в следующей ячейке.



In [ ]:
print ('Список категорий')
feature_list = ['Одежда (также нужно искать "одежды")', 'Аптеки', 'Ценные бумаги', 'Магазины косметики и Салоны красоты', 'Образовательные услуги (а также школы, университеты)',
                'Алименты', 'Прокат автомобилей', 'Автозапчасти, Автошины и Станции техобслуживания (а также СТО)', 'Видеоигры (искать надо по "видеоигр")',
                'Отели', 'Программирование', 'Спиртное и алкоголь', 'Предсказатели будущего', 'Азартные игры']
if (int(str(v)[4]) % 2) == 0:
  print (feature_list[::2])
else:
  print (feature_list[::-2])


if (int(str(v)[5]) % 2) == 0:
  print ('В качестве значений признаков используйте количество транзакций с тратами и количество транзакций с поступлениями')
else:
  print ('В качестве значений признаков используйте сумму по транзакциям с тратами и сумму по транзакциям с поступлениями')

Список категорий
['Азартные игры', 'Спиртное и алкоголь', 'Отели', 'Автозапчасти, Автошины и Станции техобслуживания (а также СТО)', 'Алименты', 'Магазины косметики и Салоны красоты', 'Аптеки']
В качестве значений признаков используйте сумму по транзакциям с тратами и сумму по транзакциям с поступлениями


Каждая категория может быть связана с одним или несколькими mcc-кодами. Например, вот несколько mcc-кодов по категории "Одежда":
* 5621; Готовая женская одежда
* 5641; Детская одежда
* 5651; Одежда для всей семьи

Таким образом получится 14 признаков, то есть семь категорий, каждая из которых даст два значения: траты и поступления. Например, категория "Одежда" даёт признаки "Одежда траты" и "Одежда поступления".

Сделайте нормализацию значений признаков так, чтобы все означения менялись от 0 до 1.

In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
tr_mcc_codes = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/tr_mcc_codes.csv", sep=";")
transactions = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/transactions.csv', sep=",")
gender_train = pd.read_csv(filepath_or_buffer = "/content/drive/MyDrive/Colab Notebooks/gender_train.csv", sep=",")

In [ ]:
searched_categories = "азартн|спирт|алкоголь|отел|автозапчаст|автошин|станции техобслуживания| сто |алимент|косметик|красот|аптек|аптеч"

In [ ]:
chosen_mcc_codes = tr_mcc_codes[tr_mcc_codes.mcc_description.str.contains(searched_categories, case=False)]
print(chosen_mcc_codes)

     mcc_code                                    mcc_description
7        3501                     Жилье — отели, мотели, курорты
37       5122             Лекарства, их распространители, аптеки
63       5532                                           Автошины
64       5533                          Автозапчасти и аксессуары
65       5541                            Станции техобслуживания
91       5813  Бары, коктейль-бары, дискотеки, ночные клубы и...
94       5912                                             Аптеки
95       5921  Магазины с продажей спиртных напитков на вынос...
116      5977                                 Магазины косметики
131      7011   Отели, мотели, базы отдыха, сервисы бронирования
135      7230                    Салоты красоты и парикмахерские
162      7995                       Транзакции по азартным играм
179      9211  Судовые выплаты, включая алименты и детскую по...


In [ ]:
chosen_transactions = transactions[transactions.mcc_code.isin(chosen_mcc_codes.mcc_code)]

In [ ]:
chosen_transactions = chosen_transactions.merge(chosen_mcc_codes, left_on='mcc_code', right_on='mcc_code')

In [ ]:
chosen_transactions['expenses'] = transactions[transactions['amount'] < 0]['amount']
chosen_transactions['receipts'] = transactions[transactions['amount'] >= 0]['amount']

In [ ]:
data = chosen_transactions.merge(gender_train, on='customer_id')

data = data.groupby(['customer_id', 'mcc_code', 'gender', 'mcc_description']).sum()
data = data.drop(columns=['tr_type', 'amount'])

<ipython-input-64-cc0c8909a336>:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  data = data.groupby(['customer_id', 'mcc_code', 'gender', 'mcc_description']).sum()


In [ ]:
data.reset_index(drop=False, inplace=True)
data.head()

,customer_id,mcc_code,gender,mcc_description,expenses,receipts
0,22899,5541,1,Станции техобслуживания,-308237.12,0.00
1,22899,5912,1,Аптеки,-22459.16,0.00
2,28753,5541,0,Станции техобслуживания,-111902.76,314428.21
3,28753,5912,0,Аптеки,-200605.21,29196.91
4,28753,5977,0,Магазины косметики,0.00,10825.31


In [ ]:
df_pivot = data.pivot_table(index='customer_id', columns='mcc_code', values=['expenses', 'receipts'], aggfunc='sum')
df_pivot.columns = ['{1}_{0}'.format(col[0], col[1]) for col in df_pivot.columns]

df_pivot.replace(0, float('nan'), inplace=True)

df_pivot = df_pivot.merge(gender_train, on='customer_id')

In [ ]:
df_pivot.head()

,customer_id,3501_expenses,5122_expenses,5532_expenses,5533_expenses,5541_expenses,5813_expenses,5912_expenses,5921_expenses,5977_expenses,...,5541_receipts,5813_receipts,5912_receipts,5921_receipts,5977_receipts,7011_receipts,7230_receipts,7995_receipts,9211_receipts,gender
0,22899,NaN,NaN,NaN,NaN,-308237.12,NaN,-22459.16,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,28753,NaN,NaN,NaN,NaN,-111902.76,NaN,-200605.21,NaN,NaN,...,314428.21,NaN,29196.91,NaN,10825.31,NaN,45852.62,NaN,NaN,0
2,42096,NaN,NaN,NaN,NaN,NaN,NaN,-3541256.24,NaN,NaN,...,NaN,NaN,2807394.72,NaN,NaN,NaN,NaN,NaN,NaN,0
3,49793,NaN,NaN,NaN,NaN,NaN,NaN,-763611.36,-8983.66,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,50940,NaN,NaN,NaN,NaN,NaN,NaN,-1563.15,-254684.60,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [ ]:
df_pivot.head()

,customer_id,3501_expenses,5122_expenses,5532_expenses,5533_expenses,5541_expenses,5813_expenses,5912_expenses,5921_expenses,5977_expenses,...,5541_receipts,5813_receipts,5912_receipts,5921_receipts,5977_receipts,7011_receipts,7230_receipts,7995_receipts,9211_receipts,gender
0,22899,NaN,NaN,NaN,NaN,-308237.12,NaN,-22459.16,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,28753,NaN,NaN,NaN,NaN,-111902.76,NaN,-200605.21,NaN,NaN,...,314428.21,NaN,29196.91,NaN,10825.31,NaN,45852.62,NaN,NaN,0
2,42096,NaN,NaN,NaN,NaN,NaN,NaN,-3541256.24,NaN,NaN,...,NaN,NaN,2807394.72,NaN,NaN,NaN,NaN,NaN,NaN,0
3,49793,NaN,NaN,NaN,NaN,NaN,NaN,-763611.36,-8983.66,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,50940,NaN,NaN,NaN,NaN,NaN,NaN,-1563.15,-254684.60,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


## Вопросы к заданию 1

1. Для каждой категории выведите соответствующие ей mcc-коды с описаниями.

In [ ]:
print(chosen_mcc_codes)

     mcc_code                                    mcc_description
7        3501                     Жилье — отели, мотели, курорты
37       5122             Лекарства, их распространители, аптеки
63       5532                                           Автошины
64       5533                          Автозапчасти и аксессуары
65       5541                            Станции техобслуживания
91       5813  Бары, коктейль-бары, дискотеки, ночные клубы и...
94       5912                                             Аптеки
95       5921  Магазины с продажей спиртных напитков на вынос...
116      5977                                 Магазины косметики
131      7011   Отели, мотели, базы отдыха, сервисы бронирования
135      7230                    Салоты красоты и парикмахерские
162      7995                       Транзакции по азартным играм
179      9211  Судовые выплаты, включая алименты и детскую по...


2. Выведите количество пустых ячеек по каждому признаку. По какому признаку отсутствует больше всего данных?

In [ ]:
print("Множество пустых ячеек по каждому признаку")
empty_cells = df_pivot.isna().sum()
print(empty_cells)

Множество пустых ячеек по каждому признаку
customer_id         0
3501_expenses    6859
5122_expenses    6939
5532_expenses    6937
5533_expenses    4854
5541_expenses    2983
5813_expenses    5907
5912_expenses    1022
5921_expenses    4240
5977_expenses    6532
7011_expenses    6220
7230_expenses    6934
7995_expenses    6909
9211_expenses    6939
3501_receipts    6892
5122_receipts    6853
5532_receipts    6728
5533_receipts    6278
5541_receipts    4935
5813_receipts    6657
5912_receipts    4399
5921_receipts    6035
5977_receipts    3760
7011_receipts    6872
7230_receipts    6281
7995_receipts    6639
9211_receipts    6934
gender              0
dtype: int64


In [ ]:
print("Признак с наибольшим числом отсутствующих данных")
max_empty_cells_col = empty_cells.idxmax()
print(max_empty_cells_col)
print("Описание mcc кода")
print(tr_mcc_codes[tr_mcc_codes.mcc_code == int(max_empty_cells_col.split('_')[0])].mcc_description)

Признак с наибольшим числом отсутствующих данных
5122_expenses
Описание mcc кода
37    Лекарства, их распространители, аптеки
Name: mcc_description, dtype: object



3. По какой категории самая большая разница между мужчинами и женщинами?

In [ ]:
data_copy = data.copy()
data_copy['amount'] = data['expenses'] + data['receipts']

mean_male = data_copy[data_copy['gender'] == 0].groupby('mcc_code')['amount'].mean()
mean_female = data_copy[data_copy['gender'] == 1].groupby('mcc_code')['amount'].mean()

difference = abs(mean_male - mean_female)

max_difference_mcc_code = difference.idxmax()

print(tr_mcc_codes[tr_mcc_codes.mcc_code == max_difference_mcc_code])

    mcc_code          mcc_description
65      5541  Станции техобслуживания


# Задание 2 (3 балла)

In [ ]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
df_scaled = pd.DataFrame(min_max_scaler.fit_transform(df_pivot), columns=df_pivot.columns)

y = df_scaled['gender']
X = df_scaled.drop(columns=['gender'])
X.replace(float('nan'), 0, inplace=True)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_data.py:473: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_data.py:474: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)


In [ ]:
import tensorflow as tf

#X = tf.convert_to_tensor(X)
#y = tf.convert_to_tensor(y)
X = np.array(X)
y = np.array(y)

In [ ]:
# print(y)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=True)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, shuffle=True)

In [ ]:
'''print(X_train.shape)
print(y_train.shape)
print(y_train)'''

'print(X_train.shape)\nprint(y_train.shape)\nprint(y_train)'

Постройте модель - многослойную нейронную сеть минимум с двумя скрытыми слоями и с dropout. Количество нейронов в каждом слое выберите самостоятельно.
Функции активации, метод оптимизации, скорость обучения, параметр для dropout - выберите самостоятельно.

Модель нужно построить с помощью keras или torch.

Настройте параметры модели.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout, Conv2D, MaxPooling2D, BatchNormalization

In [ ]:
model = Sequential()

model.add(Flatten(input_shape=(27,)))
model.add(Activation('relu'))
model.add(Dense(units=512, input_shape=(27,)))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(Dense(units=256))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(Dense(units=128))
model.add(Activation('relu'))
model.add(Dense(units=64))
model.add(Activation('relu'))
model.add(Dense(units=32))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(Dense(units=2))
model.add(Activation('softmax', name="probabilities"))

In [ ]:
model.compile(optimizer = 'adam', loss='sparse_categorical_crossentropy', metrics = ['accuracy', 'mse'])

In [ ]:
history = model.fit(X_train, y_train,
          batch_size=128,
          epochs=35, #epochs=10,
          verbose=1,
          validation_data=(X_val, y_val))

Epoch 1/35
31/31 [==============================] - 3s 19ms/step - loss: 0.6235 - accuracy: 0.6705 - mse: 0.2815 - val_loss: 0.5906 - val_accuracy: 0.7064 - val_mse: 0.3023
Epoch 2/35
31/31 [==============================] - 0s 9ms/step - loss: 0.5955 - accuracy: 0.6974 - mse: 0.2977 - val_loss: 0.5931 - val_accuracy: 0.6964 - val_mse: 0.3034
Epoch 3/35
31/31 [==============================] - 0s 9ms/step - loss: 0.5911 - accuracy: 0.7010 - mse: 0.2979 - val_loss: 0.5871 - val_accuracy: 0.7056 - val_mse: 0.2989
Epoch 4/35
31/31 [==============================] - 0s 10ms/step - loss: 0.5897 - accuracy: 0.7010 - mse: 0.2980 - val_loss: 0.5906 - val_accuracy: 0.7118 - val_mse: 0.2836
Epoch 5/35
31/31 [==============================] - 0s 9ms/step - loss: 0.5863 - accuracy: 0.7043 - mse: 0.2993 - val_loss: 0.5867 - val_accuracy: 0.7064 - val_mse: 0.2993
Epoch 6/35
31/31 [==============================] - 0s 9ms/step - loss: 0.5890 - accuracy: 0.6992 - mse: 0.3011 - val_loss: 0.5848 - val_a

## Вопросы к заданию 2

1. Перечислите все гиперпараметры и их значения вот в таком формате (значения приведены просто для примера, у вас могут быть другие):

* количество эпох: 35
* функция активация на скрытых слоях: ReLU
* Процент дропаута: 10
* размер батча: 128


2. Сколько у вашей модели параметров?

# Задание 3 (2 балла)

Оцените качество модели не менее, чем на трех разных метриках.

In [ ]:
model.evaluate(X_test, y_test)

55/55 [==============================] - 0s 4ms/step - loss: 0.5731 - accuracy: 0.7107 - mse: 0.3122


[0.573117196559906, 0.710662841796875, 0.31217890977859497]

## Вопросы к заданию 3

1. Выведите значения метрик.

loss: 0.5731 - accuracy: 0.7107 - mse: 0.3122

2. Как улучшить модель?

Очистить входные данные от большого количества нулевых значений, использовать регуляризацию, попробовать использовать leakyReLU или elu вместо relu, подобрать лучшее значение гиперпараметров

3. Для чего может понадобиться предсказание пола клиента по его транзакциям?

Для таргетирования рекламы, прогнозирования спроса

После завершения контрольной работы дайте ссылку на colab в отдельном followup **в своей теме на Piazza**. Дайте разрешение на его просмотр.

**Не создавайте новую тему, не присылайте решение файлом - только ссылкой.**

Не забудьте запустить код в последней ячейке для контроля времени выполнения.

In [ ]:
current_datetime = datetime.now()
print(current_datetime)

2023-12-15 08:23:27.196725
